In [6]:
import os
import datetime

In [7]:
#!rm -r $tar_path
#!rm -r $bin_path

In [8]:
tar_path = '/home/kinoshita/synthetic/tar'
bin_path = '/home/kinoshita/synthetic/bin'
%mkdir -p $tar_path
%mkdir -p $bin_path
%ls /home/kinoshita/synthetic
%cd ~/synthetic/tar

bin/  tar/
/home/kinoshita/synthetic/tar


In [9]:
def get_bin_file(dt):
    # URLを指定
    directory = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original'
    date = dt.strftime('%Y/%m/%d')
    tar_filename = 'Z__C_RJTD_'+dt.strftime('%Y%m%d%H%M%S')+'_RDR_JMAGPV__grib2.tar'
    URL = directory +'/'+ date +'/'+ tar_filename

    # binファイルがあるかを確認
    bin_file_path = bin_path+'/Z__C_RJTD_'+dt.strftime('%Y%m%d%H%M%S')+'_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin'
    if os.path.exists(bin_file_path):
        print('already exist: ',bin_file_path)
    else:
        # tarファイルを~/tarに保存
        out = !wget -nc -P $tar_path $URL
        if '保存完了' in out[-2]:
            # tarファイルから1kmメッシュデータのみを取り出す
            filelist = !tar -tf $tar_filename  # tarファイル内ファイル名リスト
            ggis_name = [s for s in filelist if 'Ggis1km' in s][0]  # ファイル名を取得
            !tar -C $bin_path -xvf $tar_filename $ggis_name # tarファイルから1kmメッシュ全国合成レーダーエコー強度GPVのみを/binに取り出す
            !rm $tar_filename  # tarファイルは不要なので削除

            # データがある絶対パスを取得
            file_path = bin_path+'/'+ggis_name
            print('saved: ',file_path)

In [10]:
start = '2017/01/01 00:00:00'
end = '2018/01/01 00:00:00'
dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')

while dt < dt_limit:
    get_bin_file(dt)
    dt = dt + datetime.timedelta(minutes=10)

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101000000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101001000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101002000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101003000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101004000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101005000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101010000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101011000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101012000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101210000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101211000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101212000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101213000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101214000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101215000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101220000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101221000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170101222000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104132000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104133000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104134000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104135000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104140000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104141000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104142000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104143000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170104144000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105215000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105220000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105221000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105222000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105223000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105224000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105225000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105230000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170105231000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107153000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107154000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107155000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107160000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107161000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107162000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107163000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107164000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170107165000_RDR_JMAGPV_Ggis1km_Prr10l

...

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204094000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204095000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204100000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204101000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204102000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204103000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204104000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204105000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170204110000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207070000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207071000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207072000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207073000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207074000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207075000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207080000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207081000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170207082000_RDR_JMAGPV_Ggis1km_Prr10l

already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209144000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209145000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209150000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209151000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209152000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209153000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209154000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209155000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
already exist:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170209160000_RDR_JMAGPV_Ggis1km_Prr10l

Z__C_RJTD_20170212131000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
saved:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170212131000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20170212132000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
saved:  /home/kinoshita/synthetic/bin/Z__C_RJTD_20170212132000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
^C


IndexError: list index out of range